**Dataset 1 :** Removed Excess dog,cat,fish,butterfly images



In [1]:
!gdown 1m1B9I5I-ucYvxOJy2X65c4rL_4ueoNKs

Downloading...
From: https://drive.google.com/uc?id=1_iDBCu211qGuRmnCrkPo8uTxJG5unLmP
To: /content/sapling_final.zip
100% 264M/264M [00:04<00:00, 53.9MB/s]


**Dataset 2 :** With Excess dog,cat,fish,butterfly images


In [ ]:
!gdown 1xNALY1-J6HoQYKYPapY_r82Bsr0Mim0W

In [ ]:
!gdown 1m1B9I5I-ucYvxOJy2X65c4rL_4ueoNKs

In [8]:
!rm -r /content/sapling_final/

In [ ]:
!unzip /content/sapling_final.zip

In [10]:
#To remove one corrupted image
!rm /content/sapling_final/train/Saplings/PE_693.jpg

In [22]:
import os

# Path to dataset folders
train_folder = "/content/sapling_final/train"
test_folder = "/content/sapling_final/test"
val_folder = "/content/sapling_final/val"

# Function to count images in a folder
def count_images(folder):
    count_per_category = {}
    for category in os.listdir(folder):
        category_folder = os.path.join(folder, category)
        if os.path.isdir(category_folder):
            images = os.listdir(category_folder)
            count_per_category[category] = len(images)
    return count_per_category

# Count images in train folder
train_counts = count_images(train_folder)

# Count images in test folder
test_counts = count_images(test_folder)

# Count images in val folder
val_counts = count_images(val_folder)

# Print the counts per category in train folder
print("Train:")
for category, count in train_counts.items():
    print(f"{category}: {count}")

# Print the counts per category in test folder
print("\nTest:")
for category, count in test_counts.items():
    print(f"{category}: {count}")

# Print the counts per category in val folder
print("\nValidation:")
for category, count in val_counts.items():
    print(f"{category}: {count}")


Train:
Saplings: 663
Non-Saplings: 2020

Test:
Saplings: 132
Non-Saplings: 404

Validation:
Saplings: 88
Non-Saplings: 269


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pretrained ResNet-18 model
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
num_features = model.fc.in_features

# Freeze pretrained weights
for param in model.parameters():
    param.requires_grad = False

# Add two more fully connected layers
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 1)
)  # Binary classification, output layer with 1 neuron

# Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=3.0, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        bce_loss = nn.BCEWithLogitsLoss()(inputs, targets.view(-1, 1).float())
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return focal_loss.mean()


In [47]:

criterion = FocalLoss()
# Define the data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a consistent size
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize image data
])

train_folder = "/content/sapling_final/train"
test_folder = "/content/sapling_final/test"
val_folder = "/content/sapling_final/val"
# Load your dataset and perform necessary transformations
train_dataset = ImageFolder(train_folder, transform=transform)
test_dataset = ImageFolder(test_folder, transform=transform)
val_dataset = ImageFolder(val_folder, transform=transform)

# Set batch size
batch_size = 16
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Set optimizer and learning rate
optimizer = optim.Adam(model.parameters(), lr=0.000001)

# Training loop
num_epochs = 30

model.to(device)
model.train()

for epoch in range(num_epochs):
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print average loss for the epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader):.5f}")

        # Validation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0

        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            predicted = torch.round(torch.sigmoid(outputs))

            total += labels.size(0)
            correct += (predicted.reshape((labels.size(0))) == labels).sum().item()

        val_accuracy = correct / total
        print(f"Validation Accuracy: {val_accuracy:.4f}")

    model.train()

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/30, Loss: 0.00956
Validation Accuracy: 0.9692
Epoch 2/30, Loss: 0.00636
Validation Accuracy: 0.9720
Epoch 3/30, Loss: 0.00698
Validation Accuracy: 0.9692
Epoch 4/30, Loss: 0.00665
Validation Accuracy: 0.9720
Epoch 5/30, Loss: 0.00711
Validation Accuracy: 0.9720
Epoch 6/30, Loss: 0.00803
Validation Accuracy: 0.9692
Epoch 7/30, Loss: 0.00823
Validation Accuracy: 0.9720
Epoch 8/30, Loss: 0.00662
Validation Accuracy: 0.9692
Epoch 9/30, Loss: 0.00736
Validation Accuracy: 0.9664
Epoch 10/30, Loss: 0.00681
Validation Accuracy: 0.9692
Epoch 11/30, Loss: 0.00658
Validation Accuracy: 0.9692
Epoch 12/30, Loss: 0.00602
Validation Accuracy: 0.9692
Epoch 13/30, Loss: 0.00772
Validation Accuracy: 0.9692
Epoch 14/30, Loss: 0.00666
Validation Accuracy: 0.9692
Epoch 15/30, Loss: 0.00521
Validation Accuracy: 0.9720
Epoch 16/30, Loss: 0.00563
Validation Accuracy: 0.9692
Epoch 17/30, Loss: 0.00502
Validation Accuracy: 0.9720
Epoch 18/30, Loss: 0.00659
Validation Accuracy: 0.9720
Epoch 19/30, Loss: 

In [44]:
# # Evaluation
# test_folder = "/content/pg"

# test_dataset = ImageFolder(test_folder, transform=transform)



# test_loader = DataLoader(test_dataset, batch_size=batch_size)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from sklearn.metrics import confusion_matrix, f1_score
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        predicted = torch.round(torch.sigmoid(outputs))

        total += labels.size(0)
        correct += (predicted.reshape((labels.size(0))) == labels).sum().item()
        # print(predicted.reshape((labels.size(0))).shape,labels.shape)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    test_accuracy = correct / total
    print(f"Test Accuracy: {test_accuracy:.4f}")

    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    print("Confusion Matrix:")
    print(cm)


Test Accuracy: 0.9757
Confusion Matrix:
[[394  10]
 [  3 129]]


In [45]:
# Calculate confusion matrix
cm = f1_score(all_labels, all_predictions)
print("f1_score:")
print(cm)


f1_score:
0.9520295202952029


In [ ]:
save_path = "sapling.h5"
torch.save(model.state_dict(), save_path)

In [39]:
model_path = 'sapling.pth'  # Path to save the model
torch.save(model.state_dict(), model_path)

In [ ]:
total

In [ ]:
!gcloud auth application-default login

In [35]:
from google.cloud import storage
client = storage.Client("skilful-voltage-300413")

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [36]:
bucket = client.get_bucket("sapling_bucket")

In [46]:
blob = bucket.blob("sapling.pth")
blob.upload_from_filename('/content/sapling.pth')

In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pretrained ResNet-18 model
model1 = models.resnet18(pretrained=True)
num_features = model1.fc.in_features

# Freeze pretrained weights
for param in model1.parameters():
    param.requires_grad = False

# Add two more fully connected layers
model1.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 1)
)  # Binary classification, output layer with 1 neuron

import os
model_path = 'sapling.pth'  # Path to the saved model
model1.load_state_dict(torch.load(model_path))
model1.eval()


  # Preprocess the input image
image_folder = '/content/pg1/pg2/'
  # Replace with the path to your input image
for images in os.listdir(image_folder):
    image_path = os.path.join(image_folder, images)
    print(images)
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to a consistent size
        transforms.ToTensor(),           # Convert images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize image data
    ])

    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)

    # Make predictions
    with torch.no_grad():
        output = model1(input_batch)

    # Convert sigmoid probabilities to binary predictions
    predicted = torch.sigmoid(output)
    predicted_labels = (predicted >= 0.5).int()

    # Get the predicted class label (0 or 1)
    predicted_class = predicted_labels.item()

    if predicted_class == 1:

      predicted_class_name = "Sapling"

    else:

      predicted_class_name = "Not a Sapling"


    print(f"Predicted class: {predicted_class_name}")
    print(predicted)


new1.jpeg
Predicted class: Not a Sapling
tensor([[0.2271]])
new10.jpeg
Predicted class: Not a Sapling
tensor([[0.4563]])
new13.jpeg
Predicted class: Not a Sapling
tensor([[0.3262]])
new8.jpeg
Predicted class: Not a Sapling
tensor([[0.4772]])
new4.jpeg
Predicted class: Not a Sapling
tensor([[0.3808]])
new7.jpeg
Predicted class: Not a Sapling
tensor([[0.2570]])
new5.jpeg
Predicted class: Not a Sapling
tensor([[0.1233]])
new2.jpeg
Predicted class: Not a Sapling
tensor([[0.1255]])
new6.jpeg
Predicted class: Not a Sapling
tensor([[0.4532]])
new3.jpeg
Predicted class: Not a Sapling
tensor([[0.1417]])
new12.jpeg
Predicted class: Not a Sapling
tensor([[0.4217]])
new11.jpeg
Predicted class: Sapling
tensor([[0.7852]])
new9.jpeg
Predicted class: Not a Sapling
tensor([[0.3237]])


In [2]:
!wget http://images.cocodataset.org/images/images_trainval2017.zip


--2023-06-21 12:41:07--  http://images.cocodataset.org/images/images_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.128.153, 52.217.129.169, 3.5.9.158, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.128.153|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-06-21 12:41:07 ERROR 404: Not Found.



In [ ]:
import requests

# Define the URL of the image to download
image_url = "http://images.cocodataset.org/zips/train2017.zip"

# Send a GET request to download the image
response = requests.get(image_url)

# Check if the request was successful
if response.status_code == 200:
    # Get the content of the response (image data)
    image_data = response.content

    # Save the image to a file
    with open("train2017.zip", "wb") as file:
        file.write(image_data)

    print("Image downloaded successfully.")
else:
    print("Failed to download image.")
